In [1]:
import requests
from bs4 import BeautifulSoup # note that the import package command is `bs4`
import csv
import numpy as np
import os.path
import pandas as pd

df = pd.read_csv('./cities.csv')

df.head()

,Unnamed: 0,city,link
0,0,Abernant,https://www.city-data.com/city/Abernant-Alabam...
1,1,Alabaster,https://www.city-data.com/city/Alabaster-Alaba...
2,2,"Albertville, AL",https://www.city-data.com/city/Albertville-Ala...
3,3,Alexander City,https://www.city-data.com/city/Alexander-City-...
4,4,"Andalusia, AL",https://www.city-data.com/city/Andalusia-Alaba...


In [2]:
one = df.loc[df['Unnamed: 0'].isin(range(0, 250))]
two = df.loc[df['Unnamed: 0'].isin(range(251, 500))]
three = df.loc[df['Unnamed: 0'].isin(range(501, 750))]
four = df.loc[df['Unnamed: 0'].isin(range(751, 1000))]
five = df.loc[df['Unnamed: 0'].isin(range(1001, 1250))]
six = df.loc[df['Unnamed: 0'].isin(range(1251, 1500))]
seven = df.loc[df['Unnamed: 0'].isin(range(1501, 1750))]
eight = df.loc[df['Unnamed: 0'].isin(range(1751, 2000))]
nine = df.loc[df['Unnamed: 0'].isin(range(2001, 2250))]
ten = df.loc[df['Unnamed: 0'].isin(range(2251, 2500))]
eleven = df.loc[df['Unnamed: 0'].isin(range(2501, 2750))]
twelve = df.loc[df['Unnamed: 0'].isin(range(2751, 3000))]
thirteen = df.loc[df['Unnamed: 0'].isin(range(3001, 3250))]
fourteen = df.loc[df['Unnamed: 0'].isin(range(3251, 3500))]
fifteen = df.loc[df['Unnamed: 0'].isin(range(3501, 3750))]
sixteen = df.loc[df['Unnamed: 0'].isin(range(3751, 4000))]
seventeen = df.loc[df['Unnamed: 0'].isin(range(4001, 4250))]
eighteen = df.loc[df['Unnamed: 0'].isin(range(4251, 4500))]
nineteen = df.loc[df['Unnamed: 0'].isin(range(4501, 4750))]

In [3]:
# one.to_csv('one.csv')
# two.to_csv('two.csv')
# three.to_csv('three.csv')
# four.to_csv('four.csv')
# five.to_csv('five.csv')
# six.to_csv('six.csv')
# seven.to_csv('seven.csv')
# eight.to_csv('eight.csv')
# nine.to_csv('nine.csv')
# ten.to_csv('ten.csv')
# eleven.to_csv('eleven.csv')
# twelve.to_csv('twelve.csv')

In [4]:
for item in nineteen['link']:
    response = requests.get(item)
    response_html = response.text

    soup = BeautifulSoup(response_html, features = 'lxml')
    print(item)

    #Population
    population_box = soup.find("section", attrs={'class': "city-population"})
    if population_box != None:
        population_num = population_box.text.split(':')[1].split()[0]
        try:
                population_change = population_box.text.split(':')[2].split()[0]
        except IndexError:
                population_change = np.nan

    else:
        population_num = np.nan
        population_change = np.nan
    
    #median age of population
    pop_age = soup.find("section", attrs={'class': "median-age"})
    if pop_age != None:
        median_age = pop_age.text.split()[2].strip()[4:]
    else:
        median_age = np.nan
    
    #median income
    income = soup.find("section", attrs={'class': "median-income"})
    if income != None:
        median_income = income.text.split(':')[1].split()[0]
    #Per capita income
        per_capita_income = income.text.split(':')[3].split()[0]
    #Median House Value
        if (income.text.split(':')[5].split()[0]) == 'over':
                med_house_val = income.text.split(':')[5].split()[1]
        else:
                med_house_val = income.text.split(':')[5].split()[0]
    else:
        median_income = np.nan
        per_capita_income = np.nan 
        med_house_val = np.nan

    #median rent
    rent = soup.find("section", attrs={'class': "median-rent"})
    if rent != None:
        median_rent = rent.text.split()[-1]
    else:
        median_rent = np.nan
    
    #cost_of_living_index
    index = soup.find("section", attrs={'class': "cost-of-living-index"})
    if index != None:
        cost_of_living_index = index.text.split(':')[1].split()[0]
    else:
        cost_of_living_index = np.nan
    
    #Poverty
    poverty = soup.find("section", attrs={'class': "poverty-level"})
    if poverty != None:
        percent_below_poverty_lvl = poverty.text.split(':')[1].split()[0]
    else:
        percent_below_poverty_lvl = np.nan
    
    #Crime
    crime = soup.find("section", attrs={'class': "crime"})
    
    if crime != None:
        crime_ind = crime.text.split('average')[1]
        crime_every_year = crime_ind.split()[1].split('.')
        crime_index_score = crime_every_year[-2][1:]+'.'+crime_every_year[-1]
    else:
        crime_every_year = np.nan
        crime_index_score = np.nan

    #Population Density
    pop_den = soup.find("section", attrs={'class': "population-density"})
    if pop_den != None:
        population_density = pop_den.text.split(':')[2].split()[0]
    else:
        population_density = np.nan
    
    # unemployment rate
    unemployment = soup.find("section", attrs={'class': "unemployment"})
    if unemployment != None:
        unemployment_percent = unemployment.text.split(':')[2].partition('%')[0]+'%'
    else: 
        unemployment_percent = np.nan
    
    #Most common industries
    ind = soup.find("section", attrs={'class': "most-common-industries"}).text.split('Most')
    if ind != ['']:
        industries = ind[1].split('Females')[1].split('\n')
        industries = [value for value in industries if value != '']
        most_common_indus_both = int(len(industries)/3)
        most_common_industries = industries[:most_common_indus_both]
    else:
        most_common_industries = np.nan
    
    #Taxes
    taxes = soup.find("section", attrs={'class': "real-estate-taxes"})
    if taxes != None:
        if taxes.text.split(':')[1].split()[0].startswith("$"):
            tax_mortgage = taxes.text.split(':')[1].split()[0]
        else:
            tax_mortgage = taxes.text.split(':')[2].split()[1][1:7]
        #tax_no_mortgage = tax[2].split()[0]
    else:
        tax_mortgage = np.nan
        #tax_no_mortgage = np.nan
    
    #Co-ordinates
    coordinates = soup.find("section", attrs={'class': "coordinates"}).text.split()
    if coordinates != None:
        Latitude = coordinates[1]+' N'
        Longitude = coordinates[4]+' W'
    else:
        Latitude = Longitude = np.nan

    
    #Avg Commute time
    education = soup.find("section", attrs={'class': "education-info"})
    if education !=  None:
        try:
            avg_commute = education.text.split('(commute): ')[1].split('\n')[0]
        except IndexError:
            avg_commute = np.nan     
    else:
        avg_commute = np.nan

    #AQI
    air_poll = soup.find("section", attrs={'class': "air-pollution"})
    if air_poll != None:
        try:
            aqi = air_poll.text.split('(AQI)')[1].split()[4]
        except IndexError:
            aqi = np.nan
    else:
        aqi = np.nan
        
    #nickname
    name = soup.find("section", attrs={'class': "city-nicknames"})
    if name != None:
        nickname = name.text.split(':')[1]
    else:
        nickname = np.nan

    file_exists = os.path.isfile('city_final.csv')

    with open('city_final.csv', 'a') as csv_file:
        headers = {'city':None, 'population':None, 'population_change':None, 'Median_Age':None,'Median_Income':None, 
                   'per_capita_Income':None,'Median_House_Value':None,'Median Rent':None,'Cost_of_Living_Index':None,
                   'Percent_below_Poverty':None,'Crime_rate':None,'Population_Density':None,'Unemployment_rate':None,
                   'Most_Common_Industries':None,'Property_taxes':None,'Laitude':None,'Longitude':None,
                   'Average_Commute_Time':None,'AQI':None, 'Nickname':None}
        writer = csv.DictWriter(csv_file, fieldnames = headers)
        if not file_exists:
                writer.writeheader()
        writer.writerows([{'city': item, 'population': population_num, 'population_change': population_change, 'Median_Age':median_age, 
                        'Median_Income':median_income,'per_capita_Income':per_capita_income ,'Median_House_Value':med_house_val,
                        'Median Rent':median_rent,'Cost_of_Living_Index':cost_of_living_index,'Percent_below_Poverty':percent_below_poverty_lvl,
                        'Crime_rate':crime_index_score,'Population_Density':population_density,
                        'Unemployment_rate':unemployment_percent,'Most_Common_Industries':most_common_industries,
                        'Property_taxes':tax_mortgage,'Laitude':Latitude,'Longitude':Longitude,'Average_Commute_Time':avg_commute,
                        'AQI':aqi,'Nickname':nickname}])

https://www.city-data.com/city/Tuckahoe-Virginia.html
https://www.city-data.com/city/Tysons-Corner-Virginia.html
https://www.city-data.com/city/University-of-Virginia-Virginia.html
https://www.city-data.com/city/Vienna-Virginia.html
https://www.city-data.com/city/Vinton-Virginia.html
https://www.city-data.com/city/Virginia-Beach-Virginia.html
https://www.city-data.com/city/Warrenton-Virginia.html
https://www.city-data.com/city/Waynesboro-Virginia.html
https://www.city-data.com/city/West-Falls-Church-Virginia.html
https://www.city-data.com/city/West-Gate-Virginia.html
https://www.city-data.com/city/West-Springfield-Virginia.html
https://www.city-data.com/city/Williamsburg-Virginia.html
https://www.city-data.com/city/Winchester-Virginia.html
https://www.city-data.com/city/Wolf-Trap-Virginia.html
https://www.city-data.com/city/Woodburn-Virginia.html
https://www.city-data.com/city/Woodlake-Virginia.html
https://www.city-data.com/city/Wyndham-Virginia.html
https://www.city-data.com/city/Wyt

https://www.city-data.com/city/Moses-Lake-Washington.html
https://www.city-data.com/city/Mount-Vernon-Washington.html
https://www.city-data.com/city/Mount-Vista-Washington.html
https://www.city-data.com/city/Mountlake-Terrace-Washington.html
https://www.city-data.com/city/Mukilteo-Washington.html
https://www.city-data.com/city/Newcastle-Washington.html
https://www.city-data.com/city/Normandy-Park-Washington.html
https://www.city-data.com/city/North-Beach-Washington.html
https://www.city-data.com/city/North-Bend-Washington.html
https://www.city-data.com/city/North-Creek-Washington.html
https://www.city-data.com/city/North-Lynnwood-Washington.html
https://www.city-data.com/city/North-Whidbey-Washington.html
https://www.city-data.com/city/Northeast-Yakima-Washington.html
https://www.city-data.com/city/Northwest-Benton-Washington.html
https://www.city-data.com/city/Northwest-Yakima-Washington.html
https://www.city-data.com/city/Oak-Bay-Washington.html
https://www.city-data.com/city/Oak-Har

In [5]:
final = pd.read_csv('./city_final.csv')
print(final.shape)
final.tail()

(1743, 20)


,city,population,population_change,Median_Age,Median_Income,per_capita_Income,Median_House_Value,Median Rent,Cost_of_Living_Index,Percent_below_Poverty,Crime_rate,Population_Density,Unemployment_rate,Most_Common_Industries,Property_taxes,Laitude,Longitude,Average_Commute_Time,AQI,Nickname
1738,https://www.city-data.com/city/Wynoochee-Washi...,"6,614.",NaN,39.0,"$58,519","$70,979","$222,765",NaN,98.3,NaN,NaN,26,8.3%,"['Public administration (11.0%)', 'Health care...","$1,306",47.09 N,123.58 W,21.6 minutes,63.0.,NaN
1739,https://www.city-data.com/city/Yakima-Washingt...,"93,667",+30.4%,32.5,"$41,121","$70,979","$168,700",$761.,94.9,16.9%,348.2,"4,654",7.0%,"['Health care (11.5%)', 'Educational services ...","$1,916",46.60 N,120.53 W,15.4 minutes,83.8.,The Palm Springs of Washington
1740,https://www.city-data.com/city/Yelm-Washington...,"9,125",+177.4%,29.5,"$62,851","$70,979","$244,134","$1,300.",109.9,13.4%,329.4,"1,621",5.4%,"['Public administration (12.1%)', 'Constructio...","$2,462",46.94 N,122.61 W,34.9 minutes,83.0.,NaN
1741,https://www.city-data.com/city/Bluefield-West-...,"9,869",-13.8%,42.3,"$40,506","$43,469","$107,638",$575.,80.4,21.6%,261.3,"1,130",5.9%,"['Health care (19.7%)', 'Educational services ...",$588,37.26 N,81.22 W,17.6 minutes,NaN,Nature's Air Conditioned City
1742,https://www.city-data.com/city/Bellevue-Wiscon...,"15,687",+32.6%,39.8,"$57,002","$59,305","$182,856",$852.,92.5,11.3%,NaN,"1,099",2.9%,"['Health care (10.1%)', 'Educational services ...","$3,198",44.46 N,87.97 W,17.1 minutes,42.8.,NaN


In [6]:
final.to_csv('anika_cities.csv')

In [2]:
import pandas as pd

df = pd.read_csv('./anika_cities.csv')
df.head()

,Unnamed: 0,city,population,population_change,Median_Age,Median_Income,per_capita_Income,Median_House_Value,Median Rent,Cost_of_Living_Index,...,Crime_rate,Population_Density,Unemployment_rate,Most_Common_Industries,Property_taxes,Laitude,Longitude,Average_Commute_Time,AQI,Nickname
0,0,https://www.city-data.com/city/Newburgh-New-Yo...,"28,363",+0.4%,27.8,"$38,968","$64,894","$176,678","$1,170.",137.0,...,446.0,"7,421",4.8%,"['Educational services (10.5%)', 'Health care ...","$5,979",41.50 N,74.02 W,25.9 minutes,78.8.,NaN
1,1,https://www.city-data.com/city/Newstead-New-Yo...,"8,594.",+2.3%,43.8,"$58,506","$64,894","$160,600",NaN,100.3,...,NaN,169,4.3%,"['Educational services (10.9%)', 'Construction...","$2,287",43.01 N,78.52 W,24.2 minutes,62.4.,NaN
2,2,https://www.city-data.com/city/Niagara-Falls-N...,"48,460",-12.8%,38.6,"$35,995","$64,894","$76,050",$665.,100.7,...,430.7,"3,448",6.6%,"['Health care (10.4%)', 'Accommodation & food ...","$2,613",43.09 N,79.02 W,19.2 minutes,62.4.,Cataract City
3,3,https://www.city-data.com/city/Niagara-New-Yor...,"8,378.",-6.7%,43.4,"$53,582","$64,894","$115,019",NaN,104.7,...,207.5,892,5.4%,"['Health care (7.6%)', 'Accommodation & food s...","$2,018",43.12 N,78.99 W,17.1 minutes,62.4.,NaN
4,4,https://www.city-data.com/city/North-Amityvill...,"17,862.",+7.8%,34.1,"$73,993","$64,894","$289,866","$1,355.",149.3,...,NaN,"7,343",3.7%,"['Health care (20.6%)', 'Finance & insurance (...","$7,920",40.70 N,73.41 W,25.7 minutes,82.7.,NaN


In [3]:
url = 'https://www.city-data.com/city/Alexandria-Virginia.html'

output = df[df['city'] == url]
output

,Unnamed: 0,city,population,population_change,Median_Age,Median_Income,per_capita_Income,Median_House_Value,Median Rent,Cost_of_Living_Index,...,Crime_rate,Population_Density,Unemployment_rate,Most_Common_Industries,Property_taxes,Laitude,Longitude,Average_Commute_Time,AQI,Nickname
1344,1344,https://www.city-data.com/city/Alexandria-Virg...,"160,035",+24.8%,37.0,"$100,530","$71,535","$573,200","$1,693.",174.6,...,123.0,"10,544",2.3%,"['Professional, scientific, technical services...","$5,484",38.82 N,77.07 W,29.9 minutes,87.8.,NaN


In [2]:
import pandas as pd

df = pd.read_csv('./anika_cities.csv')
df.head()

,Unnamed: 0,city,population,population_change,Median_Age,Median_Income,per_capita_Income,Median_House_Value,Median Rent,Cost_of_Living_Index,...,Crime_rate,Population_Density,Unemployment_rate,Most_Common_Industries,Property_taxes,Laitude,Longitude,Average_Commute_Time,AQI,Nickname
0,0,https://www.city-data.com/city/Newburgh-New-Yo...,"28,363",+0.4%,27.8,"$38,968","$64,894","$176,678","$1,170.",137.0,...,446.0,"7,421",4.8%,"['Educational services (10.5%)', 'Health care ...","$5,979",41.50 N,74.02 W,25.9 minutes,78.8.,NaN
1,1,https://www.city-data.com/city/Newstead-New-Yo...,"8,594.",+2.3%,43.8,"$58,506","$64,894","$160,600",NaN,100.3,...,NaN,169,4.3%,"['Educational services (10.9%)', 'Construction...","$2,287",43.01 N,78.52 W,24.2 minutes,62.4.,NaN
2,2,https://www.city-data.com/city/Niagara-Falls-N...,"48,460",-12.8%,38.6,"$35,995","$64,894","$76,050",$665.,100.7,...,430.7,"3,448",6.6%,"['Health care (10.4%)', 'Accommodation & food ...","$2,613",43.09 N,79.02 W,19.2 minutes,62.4.,Cataract City
3,3,https://www.city-data.com/city/Niagara-New-Yor...,"8,378.",-6.7%,43.4,"$53,582","$64,894","$115,019",NaN,104.7,...,207.5,892,5.4%,"['Health care (7.6%)', 'Accommodation & food s...","$2,018",43.12 N,78.99 W,17.1 minutes,62.4.,NaN
4,4,https://www.city-data.com/city/North-Amityvill...,"17,862.",+7.8%,34.1,"$73,993","$64,894","$289,866","$1,355.",149.3,...,NaN,"7,343",3.7%,"['Health care (20.6%)', 'Finance & insurance (...","$7,920",40.70 N,73.41 W,25.7 minutes,82.7.,NaN


In [13]:
df['population'] = df['population'].str.replace(r"[a-zA-Z(),/\$.]","")
df['population'] = df['population'].astype(int)


In [14]:
df.head()

,Unnamed: 0,city,population,population_change,Median_Age,Median_Income,per_capita_Income,Median_House_Value,Median Rent,Cost_of_Living_Index,...,Crime_rate,Population_Density,Unemployment_rate,Most_Common_Industries,Property_taxes,Laitude,Longitude,Average_Commute_Time,AQI,Nickname
0,0,https://www.city-data.com/city/Newburgh-New-Yo...,28363,+0.4%,27.8,"$38,968","$64,894","$176,678","$1,170.",137.0,...,446.0,"7,421",4.8%,"['Educational services (10.5%)', 'Health care ...","$5,979",41.50 N,74.02 W,25.9 minutes,78.8.,NaN
1,1,https://www.city-data.com/city/Newstead-New-Yo...,8594,+2.3%,43.8,"$58,506","$64,894","$160,600",NaN,100.3,...,NaN,169,4.3%,"['Educational services (10.9%)', 'Construction...","$2,287",43.01 N,78.52 W,24.2 minutes,62.4.,NaN
2,2,https://www.city-data.com/city/Niagara-Falls-N...,48460,-12.8%,38.6,"$35,995","$64,894","$76,050",$665.,100.7,...,430.7,"3,448",6.6%,"['Health care (10.4%)', 'Accommodation & food ...","$2,613",43.09 N,79.02 W,19.2 minutes,62.4.,Cataract City
3,3,https://www.city-data.com/city/Niagara-New-Yor...,8378,-6.7%,43.4,"$53,582","$64,894","$115,019",NaN,104.7,...,207.5,892,5.4%,"['Health care (7.6%)', 'Accommodation & food s...","$2,018",43.12 N,78.99 W,17.1 minutes,62.4.,NaN
4,4,https://www.city-data.com/city/North-Amityvill...,17862,+7.8%,34.1,"$73,993","$64,894","$289,866","$1,355.",149.3,...,NaN,"7,343",3.7%,"['Health care (20.6%)', 'Finance & insurance (...","$7,920",40.70 N,73.41 W,25.7 minutes,82.7.,NaN


In [15]:
df['population'] = df['population'].astype(int)

In [19]:
fifty = df[df['population'] >= 50000]
fifty.shape

(199, 21)

In [17]:
fifty.head()

,Unnamed: 0,city,population,population_change,Median_Age,Median_Income,per_capita_Income,Median_House_Value,Median Rent,Cost_of_Living_Index,...,Crime_rate,Population_Density,Unemployment_rate,Most_Common_Industries,Property_taxes,Laitude,Longitude,Average_Commute_Time,AQI,Nickname
13,13,https://www.city-data.com/city/North-Hempstead...,226322,+1.7%,42.4,"$121,658","$64,894","$686,309",NaN,174.4,...,NaN,"4,224",3.1%,"['Health care (13.3%)', 'Educational services ...","$6,204",40.78 N,73.67 W,34.0 minutes,96.2.,NaN
66,66,https://www.city-data.com/city/Queens-New-York...,2277251,NaN,35.4,"$64,509","$64,894","$545,800",NaN,176.7,...,NaN,"20,847",3.7%,"['Health care (11.0%)', 'Finance & insurance (...","$1,523",40.71 N,73.82 W,41.4 minutes,96.7.,NaN
67,67,https://www.city-data.com/city/Ramapo-New-York...,126595,+16.2%,28.9,"$78,781","$64,894","$413,425",NaN,165.8,...,52.0,"2,067",3.1%,"['Health care (13.8%)', 'Educational services ...","$6,184",41.12 N,74.08 W,33.0 minutes,99.0.,NaN
72,72,https://www.city-data.com/city/Rochester-New-Y...,208046,-5.3%,33.3,"$33,588","$64,894","$82,400",$833.,102.7,...,406.9,"5,806",5.5%,"['Health care (17.2%)', 'Educational services ...","$2,600",43.17 N,77.61 W,19.2 minutes,64.8.,NaN
90,90,https://www.city-data.com/city/Schenectady-New...,65625,+6.2%,38.1,"$42,565","$64,894","$105,900",$864.,105.8,...,422.4,"6,050",4.6%,"['Health care (14.6%)', 'Educational services ...","$4,243",42.80 N,73.93 W,19.5 minutes,67.6.,"\r\nElectric City, The City That Lights And Ha..."


In [21]:
fifty.columns

Index(['Unnamed: 0', 'city', 'population', 'population_change', 'Median_Age',
       'Median_Income', 'per_capita_Income', 'Median_House_Value',
       'Median Rent', 'Cost_of_Living_Index', 'Percent_below_Poverty',
       'Crime_rate', 'Population_Density', 'Unemployment_rate',
       'Most_Common_Industries', 'Property_taxes', 'Laitude', 'Longitude',
       'Average_Commute_Time', 'AQI', 'Nickname'],
      dtype='object')